# Statistical analysis of a fracture network

This notebook will show how to perform statistical analysis of the analyized network. It will show how to:
+ Fit different distributions to the dataset
+ Plot summary plots for each fitted distribution
+ Visually compare different fits using PIT
+ Show and export statistical summary tables

In [1]:
from fracability.examples import data  # import the path of the sample data
from fracability import Entities, Plotters  # import the Entities class
from fracability.operations import Statistics

import scipy.stats as ss
import matplotlib.pyplot as plt
# The following is only for jupyter to avoid matplotlib inline plots
%matplotlib qt5

## Import the Pontrelli quarry Set a and calculate the topology

In [2]:
pontrelli_data = data.Pontrelli()
data_dict = pontrelli_data.data_dict  # Get dict of paths for the data

# Create the fractures and boundary objects. 
set_a = Entities.Fractures(shp=data_dict['Set_a.shp'], set_n=1)  # to add your data put the absolute path of the shp file

boundary = Entities.Boundary(shp=data_dict['Interpretation_boundary.shp'], group_n=1)

fracture_net = Entities.FractureNetwork()

fracture_net.add_fractures(set_a)
fracture_net.add_boundaries(boundary)

fracture_net.calculate_topology()




Calculating intersections: 1944/1945




## NetworkFitter 

The network fitter class is responsible of running the statistical analysis on the fracture network. There are different options:
1. use_survival: Boolean flag to use survival (True) or treat the data as if there were no censoring (False). Default is True. 
2. complete_only: Boolean flag to use only complete measurements (True) or all the dataset (False). This flag is used only when use_survival is False. Default is False.
3. use_AIC: Boolean flag to use AIC (true) or AICc (false) for model selection. Default is True


These options are useful to compare different ways of fitting the data with survival analysis however we strongly suggest to always use survival analysis since in case of no censoring the final results will be the same as the other methods.

In [3]:
fitter = Statistics.NetworkFitter(fracture_net)

In [ ]:
print(fitter.net_data.data)
print(fitter.net_data.censoring_percentage)  # get the % of censored values
print(fitter.net_data.mean) # get the sample mean (i.e. ignoring censoring) 
print(fitter.net_data.std) # get the sample std
print(fitter.net_data.mode)  # get the sample mode
# ... See docs for the full list of sample statitistics

We can also estimate the empirical cumulative or survival curve using Kaplan Meier and compare it to a ecdf calculated on a "ignoring censoring" dataset (i.e. all the lentghs without censoring flag)

In [ ]:
KM = fitter.net_data.ecdf
lengths = fitter.network_data.lengths  # get only the length data (without knowing which one are censored or not)
ignore_ecdf = ss.ecdf(lengths).cdf
plt.plot(lengths, KM, '-k')
plt.plot(lengths, ignore_ecdf.probabilities, '--k')
plt.show()

### Fit different distributions

All the rv_continous distribution present in scipy are valid (https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions).

Each time a fit is run the Akaike, Kolmogorov-Smirnov, Koziol and Green and Anderson Darling distances are calculated and saved.

In [4]:
fitter.fit('lognorm')
fitter.fit('expon')
fitter.fit('norm')
fitter.fit('gengamma')
fitter.fit('powerlaw')
fitter.fit('weibull_min')

### Plot the different models using PIT and show the model rank table

In [ ]:
Plotters.matplot_stats_uniform(fitter)

In [ ]:
fitter.fit_records(sort_by='Akaike').iloc[:,:-1] # the iloc is to remove the last column that is not useful in this case

This table can also be saved as csv, excel or directly to clipboard in a excel friendly format 

In [ ]:
fitter.fit_result_to_csv('test_export.csv')
fitter.fit_result_to_excel('test_export.xlsx')
fitter.fit_result_to_clipboard()

### Plot a summary plot of the best ranking model

In [5]:
#Plotters.matplot_stats_summary(fitter, best=True,sort_by='Mean_rank')
Plotters.matplot_stats_summary(fitter, best=False,sort_by='Mean_rank') # plot them all